# Lesson 13 - metaclasses, protocols

## metaclass type

In Python, classes are objects too, and just like objects are instances of classes, classes themselves are instances of metaclasses.

In [1]:
class Test: pass

print(type(Test))

<class 'type'>


A metaclass is a class that defines the behavior of other classes. It is responsible for creating and controlling the class objects. When you define a class in Python, the class definition is actually passed to a ```type``` metaclass, which then creates the class object based on the provided definition.

In [4]:
class Test:

    test_attr = 10

    def test_method(self):
        print(self.test_attr)

The example above can roughly be translated to something like:

In [3]:
# Creating a new class using type as a function (like a constructor)
Test = type('Test', (object,), {'test_attr': 10, 'test_method': lambda self: print(self.test_attr)})

# Creating an instance of the class
obj = Test()

# Accessing class attributes and methods
obj.test_method()

10


In this example, we use the type metaclass to create a new class dynamically. The type constructor takes three arguments:

The first argument is the name of the class as a string, which is 'Test' in this case.

The second argument is a tuple of base classes. In this example, we specify (object,) to indicate that Test inherits from the object class, which is the default base class in Python.

The third argument is a dictionary representing the class namespace. It contains the attributes and methods of the class. In this example, we define an attribute test_attr with a value of 10 and a method test_method using a lambda function.

The type constructor returns a new class object based on the provided arguments. We assign this class object to the variable Test.

## custom metaclasses

You can define your own metaclasses to modify the class creation process and add custom behavior to classes. Metaclasses allow you to perform tasks such as:

Modifying the class definition before the class is created
Injecting additional attributes or methods into the class
Implementing class-level validation or constraints
Automatically registering classes in a registry
Generating code or modifying the class structure dynamically
To define a metaclass, you create a class that inherits from type. The metaclass can define special methods such as ```__new__``` and ```__init__``` to customize the class creation process. When you define a class and specify the metaclass using the metaclass keyword argument, Python calls the metaclass's ```__new__``` method to create the class object and then calls the ```__init__``` method to initialize it. On a higher level there is also a ```__call__``` which being called each time a new instance is requested and its role is to call the ```__new__``` and ```__init``` one by one.

In [8]:
class DocstringMetaclass(type):

    def __new__(cls, name, bases, attrs):
        for name, value in attrs.items():
            # check if the attr is a func and it has a docstr
            if callable(value) and value.__doc__ is None:
                raise TypeError(f"Method '{name}' in class '{name}' must have a docstring.")
        return super().__new__(cls, name, bases, attrs)

class MyClass(metaclass=DocstringMetaclass):

    def my_method(self):
        pass  # No docstring - will raise an error
        # try to modify it and add a docstring

    def another_method(self):
        """This method has a docstring."""
        pass

TypeError: Method 'my_method' in class 'my_method' must have a docstring.

In the example above, we define a metaclass called DocstringMetaclass. The ```__new__``` method of the metaclass is responsible for creating the class object. Inside ```__new__```, we iterate over the attributes of the class using the attrs dictionary.

For each attribute, we check if it is a callable (i.e., a method) using the ```callable()``` function. If the attribute is a method, we check the presence of the ```__doc__``` attribute.

If a method is found without a docstring, we raise a TypeError with an appropriate error message indicating which method is missing a docstring.

If all methods have docstrings, the metaclass allows the class creation to proceed by calling the ```__new__``` method of the parent metaclass (```super().__new__```) with the provided arguments (which is basically a call to the standard ```type``` metaclass).

Another popular example is a singleton:

In [9]:
class SingletonMeta(type):
    _instances = {} #store a single instance per each class right in the metaclass
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    pass

a = Logger()
b = Logger()
print(a is b) # it's the same instance

True


The ```SingletonMeta``` metaclass maintains a dictionary called _instances to keep track of the instances of each class. The ```__call__``` method of the metaclass is invoked whenever an instance of the class is created.

Inside the ```__call__``` method, we check if an instance of the class already exists in the _instances dictionary. If it doesn't exist, we create a new instance using ```super().__call__(*args, **kwargs)```, which calls the ```__call__``` method of the parent metaclass (usually type) to create the instance. We then store the newly created instance in the _instances dictionary with the class as the key.

If an instance of the class already exists in the _instances dictionary, we simply return that instance instead of creating a new one.

The singleton pattern is useful in scenarios where you want to ensure that only one instance of a class exists throughout the lifetime of the program. It can be used to manage global resources, configuration settings, or any other objects that should have a single instance.

By using a metaclass to implement the singleton pattern, we can enforce the singleton behavior at the class level, making it transparent to the users of the class. The metaclass takes care of creating and managing the single instance behind the scenes.

## protocols

In Python, protocols are a way to define a set of methods and behaviors that a class should implement in order to be considered a certain type of object. Protocols are not explicitly enforced by the language itself, but rather they are a convention or an agreement that classes can follow to provide a specific interface.

Python uses duck typing, which means that an object's suitability for a particular operation is determined by the presence of certain methods or attributes, rather than by its explicit type or inheritance. Protocols leverage this concept to define a common interface that classes can adhere to.

There are several built-in protocols in Python that classes can implement to exhibit certain behaviors. Some common protocols include:

- Iterable Protocol:

Classes that implement the ```__iter__()``` and ```next()``` methods are considered iterable.
Objects of such classes can be used in for loops and with functions like ```iter()``` and ```next()```.
Examples of iterable objects include lists, tuples, strings, and custom classes that define ```__iter__()```.

- Sequence Protocol:

Classes that implement the ```__len__()``` and ```__getitem__()``` methods are considered sequences.
Sequences are iterable and support indexing and slicing operations.
Examples of sequence objects include lists, tuples, and strings.

- Mapping Protocol:

Classes that implement the ```__getitem__()```, ```__setitem__()```, ```__delitem__()```, and ```keys()``` methods are considered mappings.
Mappings are collections of key-value pairs and support item access using square bracket notation (```[]```).
Examples of mapping objects include dictionaries and custom classes that define the required methods.

- Callable Protocol:

Classes that implement the ```__call__()``` method are considered callable.
Objects of such classes can be invoked like functions using parentheses (```()```).
Examples of callable objects include functions, methods, and custom classes that define ```__call__()```.

- Context Manager Protocol:

Classes that implement the ```__enter__()``` and ```__exit__()``` methods are considered context managers.
Context managers are used with the with statement to define a block of code with setup and cleanup actions.
Examples of context manager objects include file objects and custom classes that define ```__enter__()``` and ```__exit__()```.
By implementing these protocols, classes can provide a consistent and expected behavior, making them compatible with various built-in functions, language constructs, and libraries that rely on these protocols.

In [10]:
class ReadOnlyFileManager:
    def __init__(self, filename):
        self.filename = filename
        self.file = None

    def __enter__(self):
        try:
            self.file = open(self.filename, 'r')
            return self.file
        except FileNotFoundError:
            print(f"File '{self.filename}' not found.")
            return None

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.file:
            self.file.close()
        if exc_type:
            print(f"An exception occurred: {exc_type}")
            return False  # Re-raise the exception
        return True


with ReadOnlyFileManager('nonexistent.txt') as file:
    if file:
        content = file.read()
        print("File content:")
        print(content)
    else:
        print("Unable to read the file.")

File 'nonexistent.txt' not found.
Unable to read the file.


n this example, we define a ReadOnlyFileManager class that implements the context manager protocol for reading files in read-only mode.

The ReadOnlyFileManager class has an ```__init__()``` method that takes the filename as a parameter and initializes the filename and file attributes.

The ```__enter__()``` method is called when the with block is entered. It tries to open the file in read-only mode using the specified filename. If the file is found, it returns the file object. If the file is not found, it prints a message indicating that the file was not found and returns None.

The ```__exit__()``` method is called when the with block is exited. It closes the file if it exists. If an exception occurred within the with block, it prints the exception type and returns False to re-raise the exception. If no exception occurred, it returns True to indicate that the exception (if any) has been handled.

## protocols or magic?

Magic methods, also known as dunder methods (double underscore methods), are special methods with double underscores before and after their names. These methods are automatically called by Python in specific situations or when certain operations are performed on objects. Magic methods allow you to define the behavior of objects in response to built-in operations and syntax.

On the other hand, protocols are a higher-level concept that define a set of methods and behaviors that a class should implement to be considered a certain type of object. Protocols are not explicitly enforced by the language itself but are rather a convention or an agreement that classes can follow to provide a specific interface.

The relationship between protocols and magic methods is that protocols often rely on the implementation of specific magic methods to achieve their desired behavior. In other words, magic methods are the building blocks that enable classes to adhere to protocols.

## Homework

Let's make a small step in a direction of strict typing. Implement a metaclass which will prevent classes from being made if they are not equipped with a certain protocol (choose any from the list in the section above)